require input "X_train_rotate.npy"
"X_test.npy"
"target_train.npy"
"test_id"

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import optimizers
from keras.optimizers import Adam, SGD
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
epochs = 30
batch_size = 20
split = 0.9

In [3]:
X_train_rotate=np.load('X_train_rotate.npy')
target_train=np.load('target_train.npy')
X_test=np.load('X_test.npy')

In [4]:
def get_callbacks(filepath, patience):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [5]:
#adam=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

def train_top_model(input_shape,drop,dense1,dense2):
    
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(dense1, activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(dense2, activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(1, activation='sigmoid'))
    mypotim=Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=mypotim,
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    #model.summary()
    return model
    
    

In [6]:
file_path = 'top_model_weights_path.h5'
#callbacks = get_callbacks(file_path,patience=5)

In [8]:
import os 
model = applications.VGG16(include_top=False, weights='imagenet')

X_train = model.predict(X_train_rotate)
input_shape=X_train.shape[1:]

#5fold cross validation
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for drop in [0.3,0.5,0.7]:
    for dense1 in [1024,512]:
        for dense2 in [512,256]:
            cvscores = []
            cvscores_train = []
            j=0
            for train, test in kfold.split(X_train, target_train):
                print('\n===================FOLD=',j)
                model = train_top_model(input_shape,drop,dense1,dense2)
                model.fit(X_train[train], target_train[train],
                           epochs=epochs,
                           batch_size=batch_size,
                           verbose=0,
                           validation_data=(X_train[test], target_train[test]),
                           callbacks=get_callbacks(file_path, patience=5))
                model.load_weights(filepath=file_path)
                scores = model.evaluate(X_train[test], target_train[test], verbose=0)
                scores_train = model.evaluate(X_train[train], target_train[train], verbose=0)
                print('training:', model.metrics_names[0], scores_train[0], model.metrics_names[1], scores_train[1])
                print('testing:', model.metrics_names[0], scores[0], model.metrics_names[1], scores[1])
                cvscores.append(scores[0])
                cvscores_train.append(scores_train[0])
                j=j+1
            
            print('drop=',drop,'dense1=',dense1,'dense2=',dense2,'Test loss average================',np.mean(cvscores), 'Test loss std:',np.std(cvscores))
    




===================FOLD= 0
training: loss 0.244580787342 acc 0.897635749561
testing: loss 0.373336014548 acc 0.815160955348

===================FOLD= 1
training: loss 0.202676654683 acc 0.904910366329
testing: loss 0.343149097847 acc 0.85254413298

===================FOLD= 2
training: loss 0.207498256132 acc 0.910366328932
testing: loss 0.309746395359 acc 0.85254413298

===================FOLD= 3
training: loss 0.217370237375 acc 0.909350649289
testing: loss 0.338682748176 acc 0.841995841996

===================FOLD= 4
training: loss 0.191604627081 acc 0.917164372968
testing: loss 0.322933845943 acc 0.849115504683
drop= 0.3 dense1= 1024 dense2= 512 Test loss average================ 0.337569620375 Test loss std: 0.021447341644

===================FOLD= 0
training: loss 0.205458445878 acc 0.916082099247
testing: loss 0.360608060598 acc 0.829698857736

===================FOLD= 1
training: loss 0.20826638645 acc 0.906988828267
testing: loss 0.33123254612 acc 0.849428868182

==============

training: loss 0.244871203951 acc 0.89815536503
testing: loss 0.338804580836 acc 0.845275181755

===================FOLD= 2
training: loss 0.227861959707 acc 0.904650558602
testing: loss 0.322893172781 acc 0.854620976178

===================FOLD= 3
training: loss 0.2101080586 acc 0.9111688312
testing: loss 0.326880704217 acc 0.836798336798

===================FOLD= 4
training: loss 0.211975410979 acc 0.91197091663
testing: loss 0.318037865901 acc 0.863683662851
drop= 0.7 dense1= 512 dense2= 256 Test loss average================ 0.334603480116 Test loss std: 0.0173200955231


In [ ]:
'from matplotlib import pyplot as plt
%matplotlib inline
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
X_test = np.load('bottleneck_features_test.npy')
predicted_test=model.predict(X_test,verbose=1)

In [ ]:
import pandas as pd
submission = pd.DataFrame()
test_id=np.load('test_id.npy')
submission['id']=test_id
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)